In [8]:
import pandas as pd

In [9]:
# import data tables
df_pm = pd.read_csv('California_Population.csv')
df_vehicles = pd.read_csv('vehicle_df')

In [10]:
# seperating creating ev/gas columns so its easier to compare with pm2.5 values 
df_ev_vehicles = df_vehicles[df_vehicles['Fuel Category'] == 'EV']
df_ev_vehicles = df_ev_vehicles.rename(columns={'Number of Vehicles': 'Number of EV'})
df_ev_vehicles = df_ev_vehicles.drop(columns=['Fuel Category'])
df_ev_vehicles

,Data Year,County,Number of EV
0,2010,Alameda,20
2,2010,Alpine,0
4,2010,Amador,1
6,2010,Butte,0
8,2010,Calaveras,0
...,...,...,...
1642,2023,Tulare,3837
1644,2023,Tuolumne,698
1646,2023,Ventura,31980
1648,2023,Yolo,6757


In [11]:
df_gas_vehicles = df_vehicles[df_vehicles['Fuel Category'] == 'Fossil Fuel']
df_gas_vehicles = df_gas_vehicles.rename(columns={'Number of Vehicles': 'Number of Gas Cars'})
df_gas_vehicles = df_gas_vehicles.drop(columns=['Fuel Category'])
df_gas_vehicles

,Data Year,County,Number of Gas Cars
1,2010,Alameda,885402
3,2010,Alpine,1041
5,2010,Amador,30480
7,2010,Butte,133169
9,2010,Calaveras,39636
...,...,...,...
1643,2023,Tulare,331744
1645,2023,Tuolumne,54369
1647,2023,Ventura,651851
1649,2023,Yolo,152560


In [12]:
# recombining df_vehicles 
df_vehicles = pd.merge(df_gas_vehicles, df_ev_vehicles, on=['Data Year', 'County'], how='inner')
df_vehicles.shape

(826, 4)

In [14]:
df_pm.shape

(870, 4)

In [7]:
# df_vehicles.shape = df_pm.shape + 60, so ...

counties = pd.DataFrame({
    'County_df_vehicles': df_vehicles['County'],
    'County_df_pm': df_pm['County']
})
counties.head()
# Get unique counties from both DataFrames
counties_vehicles = set(df_vehicles['County'])
counties_pm = set(df_pm['County'])

# Find differences
only_in_vehicles = counties_vehicles - counties_pm
only_in_pm = counties_pm - counties_vehicles

print("Only in df_vehicles:", only_in_vehicles)
print("Only in df_pm:", only_in_pm)

# do we want to drop these values ? i will merge on left for now, idk if we will need these since they don't have pm2.5 data at all. we could estimate these values by comparing the counties around them that do report pm2.5 values. 

Only in df_vehicles: {'Kings', 'Colusa', 'Sonoma', 'Amador', 'Mendocino', 'Yuba', 'Orange', 'Plumas', 'Siskiyou', 'Stanislaus', 'San Diego', 'Alameda', 'Modoc', 'Glenn', 'Sierra', 'Trinity', 'Alpine', 'Mariposa', 'Out Of State', 'Marin', 'Sutter', 'Imperial', 'San Bernardino', 'Santa Cruz', 'San Benito', 'San Mateo', 'Tulare', 'Butte', 'Contra Costa', 'Mono', 'Napa', 'Santa Barbara', 'Fresno', 'Placer', 'Inyo', 'Humboldt', 'El Dorado', 'San Luis Obispo', 'Tehama', 'Lake', 'Shasta', 'Riverside', 'Calaveras', 'Ventura', 'Santa Clara', 'Lassen', 'Yolo', 'Monterey', 'Kern', 'Los Angeles', 'Solano', 'Tuolumne', 'Nevada', 'Merced', 'Madera'}
Only in df_pm: {'Mono                ', 'El Dorado           ', 'Kern                ', 'Placer              ', 'Tuolumne            ', 'Inyo                ', 'Santa Cruz          ', 'Shasta              ', 'State Total', 'Ventura             ', 'Marin               ', 'Siskiyou            ', 'San Bernardino    ', 'Contra Costa        ', 'Solano        

In [8]:
# merging pm and vehicle datatables
df_pm_vehicles = pd.merge(df_pm, df_vehicles, on=['Data Year', 'County'], how='left')
df_pm_vehicles

,County,Data Year,Population,Z Score,Number of Gas Cars,Number of EV
0,Alameda,2010,1510271,0.033519,NaN,NaN
1,Amador,2010,38091,-0.251576,NaN,NaN
2,Butte,2010,220000,-0.216348,NaN,NaN
3,Calaveras,2010,45578,-0.250126,NaN,NaN
4,Colusa,2010,21419,-0.254805,NaN,NaN
...,...,...,...,...,...,...
865,Tulare,2024,478918,-0.166208,NaN,NaN
866,Tuolumne,2024,54407,-0.248416,NaN,NaN
867,Ventura,2024,823863,-0.099407,NaN,NaN
868,Yolo,2024,221666,-0.216026,NaN,NaN


In [9]:
df_pm_vehicles.to_csv('/Users/clarissaclark/SIADS_Milestone_1/PM2.5_Vehicles.csv', index=False)

In [10]:
# now let's combine that dataframe with population 
df_pop = pd.read_csv('/Users/clarissaclark/SIADS_Milestone_1/California_Population.csv') 
# clean df_pop county column to match df_pm_vehicle county column 
df_pop['County'] = df_pop['County'].str.strip()
df_pop['County'] = df_pop['County'].str.title()
df_pop['County'] = df_pop['County'].str.replace(r'\s+', ' ', regex=True) 

In [11]:
# Get unique counties from both DataFrames
counties_pop = set(df_pop['County'])
counties_pm_vehicles = set(df_pm_vehicles['County'])

# Find differences
only_in_pop = counties_pop - counties_pm_vehicles
only_in_pm_vehicles = counties_pm_vehicles - counties_pop

print("Only in df_pop:", only_in_pop)
print("Only in df_pm_vehicles:", only_in_pm_vehicles)
# for now, we will merge on left for df_pm_vehicles 

Only in df_pop: {'Kings', 'Colusa', 'Sonoma', 'Amador', 'Mendocino', 'Yuba', 'Orange', 'Plumas', 'Siskiyou', 'Stanislaus', 'San Diego', 'Alameda', 'Modoc', 'Glenn', 'Sierra', 'Trinity', 'Mariposa', 'Marin', 'Sutter', 'Imperial', 'San Bernardino', 'Santa Cruz', 'San Benito', 'San Mateo', 'Tulare', 'Butte', 'Contra Costa', 'Mono', 'Napa', 'Santa Barbara', 'Fresno', 'Placer', 'Inyo', 'Humboldt', 'El Dorado', 'San Luis Obispo', 'Tehama', 'Lake', 'Shasta', 'Riverside', 'Madera', 'Calaveras', 'Ventura', 'Santa Clara', 'Lassen', 'Yolo', 'Monterey', 'Los Angeles', 'Solano', 'Tuolumne', 'Nevada', 'Merced', 'Kern'}
Only in df_pm_vehicles: {'Mono                ', 'El Dorado           ', 'Kern                ', 'Placer              ', 'Tuolumne            ', 'Inyo                ', 'Santa Cruz          ', 'Shasta              ', 'Ventura             ', 'Marin               ', 'Siskiyou            ', 'San Bernardino    ', 'Contra Costa        ', 'Solano              ', 'Amador              ', 'Men

In [12]:
df_pm_vehicles_pop= pd.merge(df_pm_vehicles, df_pop, on=['County', 'Data Year'], how='left')
df_pm_vehicles_pop.rename(columns={'Z Score':'Population Z Score'}, inplace=True)
df_pm_vehicles_pop

,County,Data Year,Population_x,Z Score_x,Number of Gas Cars,Number of EV,Population_y,Z Score_y
0,Alameda,2010,1510271,0.033519,NaN,NaN,NaN,NaN
1,Amador,2010,38091,-0.251576,NaN,NaN,NaN,NaN
2,Butte,2010,220000,-0.216348,NaN,NaN,NaN,NaN
3,Calaveras,2010,45578,-0.250126,NaN,NaN,NaN,NaN
4,Colusa,2010,21419,-0.254805,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
865,Tulare,2024,478918,-0.166208,NaN,NaN,NaN,NaN
866,Tuolumne,2024,54407,-0.248416,NaN,NaN,NaN,NaN
867,Ventura,2024,823863,-0.099407,NaN,NaN,NaN,NaN
868,Yolo,2024,221666,-0.216026,NaN,NaN,NaN,NaN


In [13]:
# to csv 
df_pm_vehicles_pop.to_csv('PM2.5_Vehicles_Population.csv', index=False)